In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
import time
import os
import joblib

from stable_baselines3 import PPO

print("MT5 Initialize:", mt5.initialize())


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
Users of this version of Gym should be able to simply replace 'import gym' with 'import gymnasium as gym' in the vast majority of cases.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


MT5 Initialize: True


In [8]:
# ------ CONFIG ------
DATA_DIR = os.path.join("data", "multiasset")      # normalized CSVs and scalers
MODEL_DIR = os.path.join("models", "multiasset")
MODEL_FILE = os.path.join(MODEL_DIR, "ppo_multiasset.zip")
SCALER_GLOB = os.path.join(DATA_DIR, "*_scaler.csv")
EMBED_FILE = os.path.join(MODEL_DIR, "asset_embeddings.npy")   # optional
ASSET_MAP_FILE = os.path.join(DATA_DIR, "asset_to_idx.csv")

In [9]:
#SCALER_DIR = "scalers/"
SCALER_DIR = SCALER_GLOB
EMB_DIR = "embeddings/"

scalers = {}
embeddings = {}

for file in os.listdir(SCALER_DIR):
    if file.endswith(".csv"):
        safe = file.replace("_scaler.csv","")
        scalers[safe] = pd.read_csv(SCALER_DIR + file, index_col=0)

for file in os.listdir(EMB_DIR):
    if file.endswith(".npy"):
        safe = file.replace(".npy","")
        embeddings[safe] = np.load(EMB_DIR + file)

safe_names = list(scalers.keys())
print("Loaded scalers:", len(scalers))
print("Loaded embeddings:", len(embeddings))


OSError: [WinError 123] The filename, directory name, or volume label syntax is incorrect: 'data\\multiasset\\*scaler.csv'

In [ ]:
MODEL_PATH = "models/ppo_multiasset.zip"
model = PPO.load(MODEL_PATH)
print("Model loaded.")


In [10]:
import os
import pandas as pd

scalers = {}

folder = "data/multiasset"
for f in os.listdir(folder):
    if f.endswith("_scaler.csv"):
        safe = f.replace("_scaler.csv", "")
        scalers[safe] = pd.read_csv(os.path.join(folder, f), index_col=0).to_dict()['mean']
        # Actually read BOTH mean and std correctly
        df = pd.read_csv(os.path.join(folder, f), index_col=0)
        scalers[safe] = {
            "mean": df["mean"],
            "std": df["std"]
        }


In [11]:
safe_names = list(scalers.keys())
print("Safe symbols:", safe_names)

Safe symbols: ['EURUSD', 'Jump_100_Index', 'Jump_10_Index', 'Jump_25_Index', 'Jump_50_Index', 'Jump_75_Index', 'Volatility_100_1s_Index', 'Volatility_100_Index', 'Volatility_10_1s_Index', 'Volatility_10_Index', 'Volatility_25_1s_Index', 'Volatility_25_Index', 'Volatility_50_1s_Index', 'Volatility_50_Index', 'Volatility_75_1s_Index', 'Volatility_75_Index']


In [12]:
def fetch_and_build_obs(symbol, window, scalers, embeddings, safe_names):

    safe_names = symbol.replace(" ", "_").replace("/", "_")

    if safe_names not in scalers:
        print(f"❌ Missing scaler for {symbol}")
        return None, None, None

    if safe_names not in embeddings:
        print(f"❌ Missing embedding for {symbol}")
        return None, None, None

    scaler = scalers[safe]
    embed_vec = embeddings[safe]

    bars = mt5.copy_rates_from_pos(symbol, mt5.TIMEFRAME_M5, 0, window + 10)

    if bars is None or len(bars) < window + 5:
        print(f"❌ Not enough bars for {symbol}")
        return None, None, None

    df = pd.DataFrame(bars)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df = df.set_index("time")
    df = df[["open","high","low","close","tick_volume"]].rename(columns={"tick_volume":"volume"})

    pct = df.pct_change().dropna()
    pct = pct.tail(window)

    if len(pct) < window:
        print(f"❌ Pct-change insufficient for {symbol}")
        return None, None, None

    pct_norm = (pct - scaler["mean"]) / scaler["std"]

    # extra features
    last_price = df["close"].iloc[-1]
    vol_est = pct["close"].std()

    balance_norm = np.full((window, 1), 1.0)
    asset_norm = safe_names.index(safe) / len(safe_names)
    asset_norm = np.full((window, 1), asset_norm)

    emb = np.tile(embed_vec, (window, 1))

    obs = np.column_stack([
        pct_norm[["open","high","low","close","volume"]].values.astype(np.float32),
        emb.astype(np.float32),
        balance_norm.astype(np.float32),
        asset_norm.astype(np.float32),
    ])

    return obs.astype(np.float32), float(vol_est), float(last_price)


In [13]:
def get_symbol_positions(symbol):
    pos = mt5.positions_get(symbol=symbol)
    return [] if pos is None else pos


def place_order(symbol, action, lot=0.10):
    """action: 1 BUY, 2 SELL"""
    price = mt5.symbol_info_tick(symbol).ask if action == 1 else mt5.symbol_info_tick(symbol).bid
    order_type = mt5.ORDER_TYPE_BUY if action == 1 else mt5.ORDER_TYPE_SELL

    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "magic": 8888,
        "comment": "RL-Trade",
        "type_filling": mt5.ORDER_FILLING_FOK
    }

    result = mt5.order_send(request)
    print("Order:", result)
    return result


def close_all_positions(symbol):
    """Close all open trades for symbol."""
    positions = get_symbol_positions(symbol)
    for p in positions:
        order_type = mt5.ORDER_TYPE_SELL if p.type == 0 else mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).bid if p.type == 0 else mt5.symbol_info_tick(symbol).ask

        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": p.volume,
            "position": p.ticket,
            "type": order_type,
            "price": price,
            "magic": 8888,
            "comment": "RL-close",
            "type_filling": mt5.ORDER_FILLING_FOK
        }
        mt5.order_send(request)


In [14]:
def execute_action(symbol, action):
    """
    action: 
      0 = HOLD
      1 = BUY
      2 = SELL
    """

    positions = get_symbol_positions(symbol)

    # If HOLD — do nothing
    if action == 0:
        print(symbol, "→ HOLD")
        return

    # Reverse handling
    if action == 1 and any(p.type == 1 for p in positions):
        # Existing SELL → close first
        print(symbol, "🔁 Reversal SELL → BUY. Closing sells.")
        close_all_positions(symbol)

    if action == 2 and any(p.type == 0 for p in positions):
        # Existing BUY → close first
        print(symbol, "🔁 Reversal BUY → SELL. Closing buys.")
        close_all_positions(symbol)

    # Enforce max 2 positions
    if len(positions) >= 2:
        print("⚠️ Max positions reached — skip")
        return

    # Place new trade
    print("📌 Executing:", symbol, "Action:", action)
    place_order(symbol, action)


In [15]:
SYMBOLS = ["EURUSD", "GBPUSD", "XAUUSD"]
WINDOW = 50
INTERVAL_SEC = 10

trade_log = []

while True:
    for symbol in SYMBOLS:

        obs, vol, price = fetch_and_build_obs(
            symbol, WINDOW, scalers, embeddings, safe_names
        )

        if obs is None:
            continue

        action, _ = model.predict(obs, deterministic=True)

        execute_action(symbol, int(action))

        trade_log.append({
            "time": pd.Timestamp.utcnow(),
            "symbol": symbol,
            "action": int(action),
            "price": price,
            "vol": vol
        })

    print("Sleeping...")
    time.sleep(INTERVAL_SEC)


❌ Missing embedding for EURUSD
❌ Missing scaler for GBPUSD
❌ Missing scaler for XAUUSD
Sleeping...
❌ Missing embedding for EURUSD
❌ Missing scaler for GBPUSD
❌ Missing scaler for XAUUSD
Sleeping...


KeyboardInterrupt: 

In [ ]:
import numpy as np

trades = pd.DataFrame(trade_log)

def evaluate_performance():
    positions = mt5.history_deals_get(
        pd.Timestamp.utcnow() - pd.Timedelta(days=5),
        pd.Timestamp.utcnow()
    )

    if positions is None:
        print("No history.")
        return

    df = pd.DataFrame(positions)

    df["profit"] = df["profit"]
    df["R"] = df["profit"] / (df["volume"] * 10)

    win_rate = (df["profit"] > 0).mean()
    profit_factor = df[df["profit"] > 0]["profit"].sum() / abs(df[df["profit"] < 0]["profit"].sum())
    avg_R = df["R"].mean()
    max_dd = df["profit"].cumsum().min()

    print("🔥 PERFORMANCE METRICS")
    print("Win-rate:", round(win_rate, 3))
    print("Profit Factor:", round(profit_factor, 3))
    print("Average R:", round(avg_R, 3))
    print("Max Drawdown:", round(max_dd, 3))
